# Full 3D Model 

In [2]:
import sympy as sp
import numpy as np
from sympy import Matrix, symbols, sin, cos, pi, simplify, pprint, init_printing

# init_printing(use_unicode=True)

### All symbols used / symbolic variables

In [ ]:
# Define symbolic variables
t = sp.Symbol('t', real=True)  # time

# Chassis state variables
x, y, z = sp.symbols('x y z', real=True, cls=sp.Function)
roll, pitch, yaw = sp.symbols('roll pitch yaw', real=True, cls=sp.Function)

# Define functions of time for chassis position and orientation
x = x(t)
y = y(t)
z = z(t)
roll = roll(t)
pitch = pitch(t)
yaw = yaw(t)

# Define derivatives
x_dot = sp.diff(x, t)
y_dot = sp.diff(y, t)
z_dot = sp.diff(z, t)
roll_dot = sp.diff(roll, t)
pitch_dot = sp.diff(pitch, t)
yaw_dot = sp.diff(yaw, t)

# Second derivatives
x_ddot = sp.diff(x_dot, t)
y_ddot = sp.diff(y_dot, t)
z_ddot = sp.diff(z_dot, t)
roll_ddot = sp.diff(roll_dot, t)
pitch_ddot = sp.diff(pitch_dot, t)
yaw_ddot = sp.diff(yaw_dot, t)

# Joint angles for left leg
theta_l1, theta_l2 = sp.symbols('theta_l1 theta_l2', real=True, cls=sp.Function)
theta_l1 = theta_l1(t)
theta_l2 = theta_l2(t)
theta_l1_dot = sp.diff(theta_l1, t)
theta_l2_dot = sp.diff(theta_l2, t)
theta_l1_ddot = sp.diff(theta_l1_dot, t)
theta_l2_ddot = sp.diff(theta_l2_dot, t)

# Joint angles for right leg
theta_r1, theta_r2 = sp.symbols('theta_r1 theta_r2', real=True, cls=sp.Function)
theta_r1 = theta_r1(t)
theta_r2 = theta_r2(t)
theta_r1_dot = sp.diff(theta_r1, t)
theta_r2_dot = sp.diff(theta_r2, t)
theta_r1_ddot = sp.diff(theta_r1_dot, t)
theta_r2_ddot = sp.diff(theta_r2_dot, t)

# Wheel angles
phi_l, phi_r = sp.symbols('phi_l phi_r', real=True, cls=sp.Function)
phi_l = phi_l(t)
phi_r = phi_r(t)
phi_l_dot = sp.diff(phi_l, t)
phi_r_dot = sp.diff(phi_r, t)
phi_l_ddot = sp.diff(phi_l_dot, t)
phi_r_ddot = sp.diff(phi_r_dot, t)

# Control inputs (torques)
tau_l1, tau_l2, tau_lw = sp.symbols('tau_l1 tau_l2 tau_lw', real=True)  # Left leg and wheel torques
tau_r1, tau_r2, tau_rw = sp.symbols('tau_r1 tau_r2 tau_rw', real=True)  # Right leg and wheel torques

# System parameters (constants)
m_c = sp.Symbol('m_c', positive=True)  # Chassis mass
m_1, m_2 = sp.symbols('m_1 m_2', positive=True)  # Mass of each link
m_w = sp.symbols('m_w', positive=True)  # Wheel masses

# Chassis dimensions
cl_x, cl_y, cl_z = sp.symbols('l_x l_y l_z', positive=True)  # Chassis dimensions

# Link lengths
L_1, L_2 = sp.symbols('L_1 L_2', positive=True)  # Link lengths
r_w = sp.Symbol('r_w', positive=True)  # Wheel radius
W_link = sp.Symbol('W_link', positive=True)  # Link width
H_link = sp.Symbol('H_link', positive=True)  # Link height
H_wheel = sp.Symbol('H_wheel', positive=True)  # Wheel height

# Gravity
g = sp.Symbol('g', positive=True)

# Define the positions of joint attachments on the chassis (in chassis frame)
lx, ly, lz = sp.symbols('lx ly lz', real=True)  # Position offsets of joint from chassis COM

# Inertia tensors
I_c = sp.Matrix(
    [[1/12*m_c*(cl_y**2 + cl_z**2), 0, 0],
     [0, 1/12*m_c*(cl_x**2 + cl_z**2), 0],
     [0, 0, 1/12*m_c*(cl_x**2 + cl_y**2)]]
) # chassis is a block
I_l1 = sp.Matrix(
    [[1/12*m_1*(W_link**2 + H_link**2), 0, 0],
     [0, 1/12*m_1*(L_1**2 + H_link**2), 0],
     [0, 0, 1/12*m_1*(L_1**2 + W_link**2)]]
) # link 1 is a block
I_l2 = sp.Matrix(
    [[1/12*m_2*(W_link**2 + H_link**2), 0, 0],
     [0, 1/12*m_2*(L_2**2 + H_link**2), 0],
     [0, 0, 1/12*m_2*(L_2**2 + W_link**2)]]
) # link 2 is a block
I_w = sp.Matrix(
    [[1/12*m_w*(3*r_w**2 + H_wheel**2), 0, 0],
     [0, 1/2*m_w*r_w**2, 0],
     [0, 0, 1/12*m_w*(3*r_w**2 + H_wheel**2)]]
) # wheel is a cylinder


# Define rotation matrices for chassis orientation
def Rot_x(angle):
    return Matrix([
        [1, 0, 0],
        [0, cos(angle), -sin(angle)],
        [0, sin(angle), cos(angle)]
    ])

def Rot_y(angle):
    return Matrix([
        [cos(angle), 0, sin(angle)],
        [0, 1, 0],
        [-sin(angle), 0, cos(angle)]
    ])

def Rot_z(angle):
    return Matrix([
        [cos(angle), -sin(angle), 0],
        [sin(angle), cos(angle), 0],
        [0, 0, 1]
    ])


### Calculate Positions

In [ ]:

# Full rotation matrix (ZYX convention)
R_chassis = Rot_z(yaw) * Rot_y(pitch) * Rot_x(roll)
R_l1 = Rot_y(theta_l1) # Rotation matrix for left leg link 1
R_l2 = Rot_y(theta_l2) # Rotation matrix for left leg link 2
R_r1 = Rot_y(theta_r1) # Rotation matrix for right leg link 1
R_r2 = Rot_y(theta_r2) # Rotation matrix for right leg link 2

# Chassis position in world frame
p_chassis = Matrix([x, y, z])

# Calculate positions of leg joints in world frame
p_left_joint = p_chassis + R_chassis * Matrix([lx, -ly, lz])
p_right_joint = p_chassis + R_chassis * Matrix([lx, ly, lz])

# Calculate positions along the left leg
p_left_link1_end = p_chassis + R_chassis * Matrix([
    lx + -L_1 * cos(theta_l1), 
    -ly, 
    lz + L_1 * sin(theta_l1)])

p_left_link2_end = p_left_link1_end + R_chassis * Matrix([
    -L_2 * cos(theta_l1 + theta_l2),
    0,
    L_2 * sin(theta_l1 + theta_l2)
])

p_left_wheel = p_left_link2_end
# Calculate positions along the right leg
p_right_link1_end = p_right_joint + R_chassis * Matrix([
    lx + L_1 * cos(theta_r1),
    ly,
    lz + L_1 * sin(theta_r1)
])

p_right_link2_end = p_right_link1_end + R_chassis * Matrix([
    L_2 * cos(theta_r1 + theta_r2),
    0,
    L_2 * sin(theta_r1 + theta_r2)
])

p_right_wheel = p_right_link2_end

# Calculate centers of mass (COM) for links (assuming COM at middle of links)
p_left_link1_com = p_left_joint + 0.5 * (p_left_link1_end - p_left_joint)
p_left_link2_com = p_left_link1_end + 0.5 * (p_left_link2_end - p_left_link1_end)
p_right_link1_com = p_right_joint + 0.5 * (p_right_link1_end - p_right_joint)
p_right_link2_com = p_right_link1_end + 0.5 * (p_right_link2_end - p_right_link1_end)


### Velocity and Kinetic Energy Calculations

In [ ]:
# Calculate velocities by taking time derivatives
v_chassis = Matrix([x_dot, y_dot, z_dot])
omega_chassis = Matrix([roll_dot, pitch_dot, yaw_dot])
omega_left_link1 = omega_chassis + R_chassis * Matrix([0, theta_l1_dot, 0])
omega_left_link2 = omega_left_link1 + R_chassis * R_l1 * Matrix([0, theta_l2_dot, 0])
omega_right_link1 = omega_chassis + R_chassis * Matrix([0, theta_r1_dot, 0])
omega_right_link2 = omega_right_link1 + R_chassis * R_l2 * Matrix([0, theta_r2_dot, 0])
omega_left_wheel = omega_left_link2 + R_chassis * Matrix([0, phi_l_dot, 0])
omega_right_wheel = omega_right_link2 + R_chassis * Matrix([0, phi_r_dot, 0])

# For each body, calculate the velocity of its COM
v_left_link1_com = sp.diff(p_left_link1_com, t)
v_left_link2_com = sp.diff(p_left_link2_com, t)
v_right_link1_com = sp.diff(p_right_link1_com, t)
v_right_link2_com = sp.diff(p_right_link2_com, t)
v_left_wheel = sp.diff(p_left_wheel, t)
v_right_wheel = sp.diff(p_right_wheel, t)

# Calculate kinetic energy terms
# Translational kinetic energy
T_chassis_trans = 0.5 * m_c * (v_chassis.dot(v_chassis))
T_left_link1_trans = 0.5 * m_1 * (v_left_link1_com.dot(v_left_link1_com))
T_left_link2_trans = 0.5 * m_2 * (v_left_link2_com.dot(v_left_link2_com))
T_right_link1_trans = 0.5 * m_1 * (v_right_link1_com.dot(v_right_link1_com))
T_right_link2_trans = 0.5 * m_2 * (v_right_link2_com.dot(v_right_link2_com))
T_left_wheel_trans = 0.5 * m_w * (v_left_wheel.dot(v_left_wheel))
T_right_wheel_trans = 0.5 * m_w * (v_right_wheel.dot(v_right_wheel))

# Rotational kinetic energy (simplified for now, I'll add inertia tensors later)
T_chassis_rot = 0.5 * omega_chassis.transpose() * R_chassis * I_c * R_chassis.transpose() * omega_chassis
T_left_link1_rot = 0.5 * omega_left_link1.transpose() * (R_chassis * R_l1) * I_l1 * (R_chassis * R_l1).transpose() * omega_left_link1
T_left_link2_rot = 0.5 * omega_left_link2.transpose() * (R_chassis * R_l1 * R_l2) * I_l2 * (R_chassis * R_l1 * R_l2).transpose() * omega_left_link2
T_right_link1_rot = 0.5 * omega_right_link1.transpose() * (R_chassis * R_r1) * I_l1 * (R_chassis * R_r1).transpose() * omega_right_link1
T_right_link2_rot = 0.5 * omega_right_link2.transpose() * (R_chassis * R_r1 * R_r2) * I_l2 * (R_chassis * R_r1 * R_r2).transpose() * omega_right_link2
T_left_wheel_rot = 0.5 * omega_left_wheel.transpose() * (R_chassis * R_l1 * R_l2) * I_w * (R_chassis * R_l1 * R_l2).transpose() * omega_left_wheel
T_right_wheel_rot = 0.5 * omega_right_wheel.transpose() * (R_chassis * R_r1 * R_r2) * I_w * (R_chassis * R_r1 * R_r2).transpose() * omega_right_wheel

# Total kinetic energy
T = (T_chassis_trans + T_chassis_rot + 
     T_left_link1_trans + T_left_link1_rot + 
     T_left_link2_trans + T_left_link2_rot + 
     T_right_link1_trans + T_right_link1_rot + 
     T_right_link2_trans + T_right_link2_rot + 
     T_left_wheel_trans + T_left_wheel_rot + 
     T_right_wheel_trans + T_right_wheel_rot)


### Potential Energy Calculations

In [ ]:

# Potential energy
V_chassis = m_c * g * z
V_left_link1 = m_1 * g * p_left_link1_com[2]
V_left_link2 = m_2 * g * p_left_link2_com[2]
V_right_link1 = m_1 * g * p_right_link1_com[2]
V_right_link2 = m_2 * g * p_right_link2_com[2]
V_left_wheel = m_w * g * p_left_wheel[2]
V_right_wheel = m_w * g * p_right_wheel[2]

# Total potential energy
V = V_chassis + V_left_link1 + V_left_link2 + V_right_link1 + V_right_link2 + V_left_wheel + V_right_wheel


### Non-Holonomic Constraints

In [ ]:
# Wheel Contact Velocities (World Frame)

# Left Wheel Contact Velocity
v_left_contact = v_left_wheel + omega_left_wheel.cross(R_chassis*R_l1*R_l2*Matrix([0, 0, -r_w])) # Velocity of contact point

# Right Wheel Contact Velocity
v_right_contact = v_right_wheel + omega_right_wheel.cross(R_chassis*R_r1*R_r2*Matrix([0, 0, -r_w])) # Velocity of contact point


# Define Wheel Directions (Body Frame)

#  Wheel direction vectors (in the chassis frame) -> point forward along the x-axis of the chassis

u_forwards = R_chassis * Matrix([1, 0, 0]) # Forward direction of chassis
u_perp = R_chassis * Matrix([0,1,0]) # Perpendicular to wheel (no lateral move -> y dir perpendicular)

# Constraint Equations

# No Sideways Slipping (dot product with perpendicular direction = 0)
f1 = v_left_contact.dot(u_perp)   # Left wheel, no lateral slip
f2 = v_right_contact.dot(u_perp)  # Right wheel, no lateral slip

# Rolling Constraint (dot product with wheel direction = r * phi_dot)
f3 = v_left_contact.dot(u_forwards) - r_w * phi_l_dot  # Left wheel rolling no slip (thus v_contact = r * phi_dot)
f4 = v_right_contact.dot(u_forwards) - r_w * phi_r_dot # Right wheel rolling no lsip

# Lagrange Multipliers
lambda_1, lambda_2, lambda_3, lambda_4 = symbols('lambda_1 lambda_2 lambda_3 lambda_4')


### Lagrangian

In [8]:
# Lagrangian
L = T - V

# Define generalized coordinates and their derivatives
q = Matrix([x, y, z, roll, pitch, yaw, theta_l1, theta_l2, theta_r1, theta_r2, phi_l, phi_r])
q_dot = Matrix([x_dot, y_dot, z_dot, roll_dot, pitch_dot, yaw_dot, 
               theta_l1_dot, theta_l2_dot, theta_r1_dot, theta_r2_dot, phi_l_dot, phi_r_dot])
q_ddot = Matrix([x_ddot, y_ddot, z_ddot, roll_ddot, pitch_ddot, yaw_ddot,
                theta_l1_ddot, theta_l2_ddot, theta_r1_ddot, theta_r2_ddot, phi_l_ddot, phi_r_ddot])

# Define generalized forces (corresponding to generalized coordinates)
Q = Matrix([0, 0, 0, 0, 0, 0, tau_l1, tau_l2, tau_r1, tau_r2, tau_lw, tau_rw])

#Constraint and Lagrange Multiplier Lists
constraints = [f1, f2, f3, f4]
lambdas = [lambda_1, lambda_2, lambda_3, lambda_4]


### Equations of Motion using Euler-Lagrange equations

In [9]:

# Euler-Lagrange equations:
# d/dt(∂L/∂q̇) - ∂L/∂q = Q
# -> equations of motion in the form: M(q)q̈ + C(q,q̇)q̇ + G(q) = B·τ
# wheel no slip is non holonomic constraint
def derive_eom_with_constraints(L, q, q_dot, Q, constraints, lambdas):
    n = len(q)
    m = len(constraints)
    eom = []

    for i in range(n):
        # d/dt(dL/dq_dot)
        term1 = sp.diff(sp.diff(L, q_dot[i]), t)
        # dL/dq
        term2 = sp.diff(L, q[i])

        # Lagrange multiplier term λ (so if f(q, q_dot) is constraint, λ x ∂f/∂qᵢ is added)
        lagrange_term = 0
        for j in range(m):
            lagrange_term += lambdas[j] * sp.diff(constraints[j], q[i])

        # Euler-Lagrange equation
        eq = term1 - term2 - Q[i] - lagrange_term
        eom.append(eq)

    # Add the constraint equations
    for constr in constraints:
        eom.append(constr)

    return Matrix(eom)



### Initial Constants and State to Linearize about

In [10]:
# Initial joint angles
theta_l1_0 = -pi/4
theta_l2_0 = 5 * pi/4
theta_r1_0 = -pi/4
theta_r2_0 = 5 * pi/4

# Initial Chassis Height
z0 = r_w + np.abs(L_1 * sin(theta_l1_0)) + np.abs(L_2 * sin(theta_l2_0))

eq_state = {
    x: 0, y: 0, z: z0,
    roll: 0, pitch: 0, yaw: 0,
    theta_l1: theta_l1_0, theta_l2: theta_l2_0,
    theta_r1: theta_r1_0, theta_r2: theta_r2_0,
    phi_l: 0, phi_r: 0,
    x_dot: 0, y_dot: 0, z_dot: 0,
    roll_dot: 0, pitch_dot: 0, yaw_dot: 0,
    theta_l1_dot: 0, theta_l2_dot: 0,
    theta_r1_dot: 0, theta_r2_dot: 0,
    phi_l_dot: 0, phi_r_dot: 0
}

zero_control_state = {
    tau_l1: 0, tau_l2: 0, tau_lw: 0,
    tau_r1: 0, tau_r2: 0, tau_rw: 0
}

# The z0, theta_l1_0, etc. would be determined based on the desired robot posture

# Define state vector for state-space representation
x_state = Matrix([
    x, y, z, roll, pitch, yaw,
    theta_l1, theta_l2, theta_r1, theta_r2, phi_l, phi_r,
    x_dot, y_dot, z_dot, roll_dot, pitch_dot, yaw_dot,
    theta_l1_dot, theta_l2_dot, theta_r1_dot, theta_r2_dot, phi_l_dot, phi_r_dot
])

# Input vector
u = Matrix([tau_l1, tau_l2, tau_lw, tau_r1, tau_r2, tau_rw])


### Equations of Motion

In [11]:
# eoms = derive_eom(L, q, q_dot, Q)
eoms = derive_eom_with_constraints(L, q, q_dot, Q, constraints, lambdas)

In [12]:
# for eq in eoms:
#     pprint(eq)
#     print()

# Write the equations to a latex file
with open('eoms.tex', 'w') as f:
    f.write(sp.latex(eoms))

### Validate EOMS

In [12]:
# Helper to substitute symbolic "var_dot and var_ddot" for the Derivative(var, t) in the equations
def replace_derivatives(expression, q, q_dot, q_ddot, replace_q, replace_q_dot, replace_q_ddot):
    """
    Replaces Derivative objects in a SymPy expression with symbolic representations.

    Args:
        expression: The SymPy expression (e.g., an equation of motion).
        q: A list of generalized coordinates (sympy.Function objects).
        q_dot: A list of symbolic representations of the first derivatives (symbols).
        q_ddot: A list of symbolic representations of the second derivatives (symbols).

    Returns:
        A new SymPy expression with Derivative objects replaced.
    """
    replacement_dict = {}

    # have to replace in reverse order to avoid matching q_dot to q_ddot

    # Second derivatives
    for i in range(len(q)):
        replacement_dict[sp.Derivative(q[i], t, t)] = replace_q_ddot[i]
        #Handle cases of diff(x_dot,t) too
        replacement_dict[sp.Derivative(q_dot[i],t)] = replace_q_ddot[i]
        
    temp_expr = expression.subs(replacement_dict)
    # print replacement_dict
    for (k, v) in replacement_dict.items():
        print(k)
        print("Replaced with")
        print(v)
        print()

    replacement_dict = {}

    # First derivatives
    for i in range(len(q)):
        replacement_dict[sp.Derivative(q[i], t)] = replace_q_dot[i]
        replacement_dict[q_dot[i]] = replace_q_dot[i]

    temp_expr = temp_expr.subs(replacement_dict)
    # print replacement_dict
    for (k, v) in replacement_dict.items():
        print(k)
        print("Replaced with")
        print(v)
        print()

    replacement_dict = {}

    # Generalized coordinates (originaly defined as functions)
    for i in range(len(q)):
        replacement_dict[sp.Derivative(q[i], t)] = replace_q[i]

    # print replacement_dict
    for (k, v) in replacement_dict.items():
        print(k)
        print("Replaced with")
        print(v)
        print()

    return temp_expr.subs(replacement_dict)
    # return expression.subs(replacement_dict)

# change q variables to a symbol instead of a function
replace_q = [sp.Symbol(f'{str(qi)[:-3]}', real=True) for qi in q]
replace_q_dot = [sp.Symbol(f'{str(qi)}_dot', real=True) for qi in replace_q]
replace_q_ddot = [sp.Symbol(f'{str(qi)}_ddot', real=True) for qi in replace_q]

In [32]:
temp_str = "x(t)"
print(f"{temp_str[:-3]}")

x


In [23]:
# replace Derivative with symbolic representations
eoms_replaced = replace_derivatives(eoms, q, q_dot, q_ddot, replace_q, replace_q_dot, replace_q_ddot)

Derivative(x(t), (t, 2))
Replaced with
x_ddot

Derivative(y(t), (t, 2))
Replaced with
y_ddot

Derivative(z(t), (t, 2))
Replaced with
z_ddot

Derivative(roll(t), (t, 2))
Replaced with
roll_ddot

Derivative(pitch(t), (t, 2))
Replaced with
pitch_ddot

Derivative(yaw(t), (t, 2))
Replaced with
yaw_ddot

Derivative(theta_l1(t), (t, 2))
Replaced with
theta_l1_ddot

Derivative(theta_l2(t), (t, 2))
Replaced with
theta_l2_ddot

Derivative(theta_r1(t), (t, 2))
Replaced with
theta_r1_ddot

Derivative(theta_r2(t), (t, 2))
Replaced with
theta_r2_ddot

Derivative(phi_l(t), (t, 2))
Replaced with
phi_l_ddot

Derivative(phi_r(t), (t, 2))
Replaced with
phi_r_ddot

Derivative(x(t), t)
Replaced with
x_dot

Derivative(y(t), t)
Replaced with
y_dot

Derivative(z(t), t)
Replaced with
z_dot

Derivative(roll(t), t)
Replaced with
roll_dot

Derivative(pitch(t), t)
Replaced with
pitch_dot

Derivative(yaw(t), t)
Replaced with
yaw_dot

Derivative(theta_l1(t), t)
Replaced with
theta_l1_dot

Derivative(theta_l2(t), t)


In [ ]:
solution = sp.solve(eoms, replace_q_ddot + lambdas, dict=True)

if not solution:
    raise ValueError("No symbolic solution found for q_ddot and lambda.")
if len(solution) > 1:
    print("Warning: Multiple symbolic solutions found. Using the first one.")
solution = solution[0]  # Use the first solution

# 4. Create expressions for q_ddot *in terms of q, q_dot, and torques*:
q_ddot_expressions = [solution[q_dd] for q_dd in replace_q_ddot]


In [ ]:
# write solution to file to save (restore later) uise pickle
import pickle as pkl
with open('solution.pkl', 'wb') as f:
    pkl.dump(solution, f)

In [39]:
# Save the replaced eoms to another file
with open('eoms_replaced.tex', 'w') as f:
    f.write(sp.latex(eoms_replaced))

In [15]:
# print(eoms_replaced)

# for every element in oems_replaced, write to a file
with open('eoms_replaced.txt', 'w') as f:
    for eq in eoms_replaced:
        f.write(str(eq))
        f.write('\n')

In [ ]:
# Substitue parameter values
# TODO Change constants to match actual robot
parameter_values = {
    m_c: 10.0,
    m_l1: 2.0,
    m_l2: 2.0,
    m_r1: 2.0,
    m_r2: 2.0,
    m_lw: 1.0,
    m_rw: 1.0,
    I_c: 5.0,
    I_l1: 0.5,
    I_l2: 0.5,
    I_r1: 0.5,
    I_r2: 0.5,
    I_lw: 0.1,
    I_rw: 0.1,
    L_1: 0.3,
    L_2: 0.3,
    r_w: 0.1,
    g: 9.81,
    lx: -0.2,
    ly: 0.1,
    lz: -0.1,
    rx: -0.2,
    ry: -0.1,
    rz: -0.1,
}
eoms_substituted = eoms_replaced.subs(parameter_values)

In [ ]:
from scipy.integrate import solve_ivp

# Create a list of variables for lambdify, in the order you'll pass them
variables = [t] + list(replace_q) + list(replace_q_dot) + [tau_l1, tau_l2, tau_lw, tau_r1, tau_r2, tau_rw]

# Lambdify the equations of motion.  'numpy' tells it to use NumPy functions.
eom_func = sp.lambdify(variables, eoms_substituted, "numpy")

In [22]:

def state_derivative(time, state, torques):
    """
    This function will be called by solve_ivp.
    
    Args:
        time: Current time.
        state:  A NumPy array containing the current values of q and q_dot.
        torques: A NumPy array containing the current torque values.
    
    Returns:
        Array containing the derivatives of q and q_dot (i.e., q_dot and q_ddot).
    """
    # Unpack the state vector into q and q_dot
    q_values = state[:len(q)]
    q_dot_values = state[len(q):]

    # Create a list of arguments for eom_func
    args = [time] + list(q_values) + list(q_dot_values) + list(torques)

    # Evaluate the EOMs
    result = eom_func(*args)

    # result is a list of equations (including constraint equations).
    # We need to extract q_dot and q_ddot. Since eom_func returns all
    # equations, including the four constraint equations, we need to handle this.
    
    #The constraints are at the very end of the output. We will make a small solver to remove them
    
    # 1. Separate Derivatives and Constraints:
    num_constraints = len(constraints)  # Number of constraint equations =4
    num_derivatives = len(q)
    derivatives = result[:num_derivatives] # First part
    constraints_eval = result[num_derivatives:]# Last part

    # 2. Set Up System for Lagrange Multipliers
    #Create symbolic q_ddot and lambda_dot for solving (these are the unknowns)
    q_ddot_sym = Matrix([sp.symbols(f'q_ddot_{i}') for i in range(num_derivatives)])
    lambda_dot_sym = Matrix([sp.symbols(f'lambda_dot_{i}') for i in range(num_constraints)])
    unknowns = list(q_ddot_sym) + list(lambda_dot_sym)

    #Replace q_ddot and lambda_dot with symbols
    equations = np.concatenate((derivatives, constraints_eval)) #All of them
    q_ddot_subs = {q_ddot[i]: q_ddot_sym[i] for i in range(len(q_ddot))} #Sub in q_ddot
    lambda_dot_subs = {sp.diff(lambdas[i],t): lambda_dot_sym[i] for i in range(len(lambdas))} #Sub in lambda_dot

    #Substitute into equations
    equations_subs = [eq.subs(q_ddot_subs).subs(lambda_dot_subs) for eq in equations]
    
    # 3. Solve for Unknowns:
    #Solve linear system
    solutions = sp.solve(equations_subs, unknowns, dict=True)

    #Check for solution
    if not solutions:
        raise ValueError("No solution found for the derivatives.")
    
    #If multiple solutions (shouldn't happen, but good to check)
    if len(solutions) > 1:
        print("Warning: Multiple solutions found, using the first one.")
    
    solution = solutions[0] #Dictionary
    
    # 4. Extract Numerical Values:
    # Extract derivatives, converting symbolic expressions to numerical values
    q_dot_num = [val for val in q_dot_values] #First part (q_dot=q_dot)
    q_ddot_num = [float(solution[q_ddot_sym[i]]) for i in range(num_derivatives)] #Sub in solution
    
    return np.array(q_dot_num + q_ddot_num, dtype=np.float64) #Return q_dot, q_ddot

# Initial conditions (example - you'll need to choose appropriate values)
initial_state = np.array([
    0.0, 0.0, 0.5,  # x, y, z
    0.0, 0.0, 0.0,  # roll, pitch, yaw
    0.1, 0.1, 0.1, 0.1, #theta
    0.0, 0.0, #phi
    0.0, 0.0, 0.0,  # x_dot, y_dot, z_dot
    0.0, 0.0, 0.0,  # roll_dot, pitch_dot, yaw_dot
    0.0, 0.0, 0.0, 0.0, #theta_dot
    0.0, 0.0
])

# Control inputs (test with constant applied torque)
torques = np.array([0.1, 0.1, 0.5, 0.1, 0.1, 0.5])

# Time span for the simulation
t_span = (0, 10)  # Simulate from 0 to 10 seconds
t_eval = np.linspace(t_span[0], t_span[1], 500)  # Evaluate at time

# Solve the differential equations
sol = solve_ivp(
    state_derivative,
    t_span,
    initial_state,
    args=(torques,),
    t_eval=t_eval,
    rtol=1e-6,
    atol=1e-8,
    dense_output=True #For smoother plots
)


TypeError: 'numpy.float64' object is not callable

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))

plt.subplot(3, 2, 1)
plt.plot(sol.t, sol.y[0, :], label='x')
plt.plot(sol.t, sol.y[1, :], label='y')
plt.plot(sol.t, sol.y[2, :], label='z')
plt.legend()
plt.title('Chassis Position')

plt.subplot(3, 2, 2)
plt.plot(sol.t, sol.y[3, :], label='roll')
plt.plot(sol.t, sol.y[4, :], label='pitch')
plt.plot(sol.t, sol.y[5, :], label='yaw')
plt.legend()
plt.title('Chassis Orientation')

plt.subplot(3, 2, 3)
plt.plot(sol.t, sol.y[6, :], label='theta_l1')
plt.plot(sol.t, sol.y[7, :], label='theta_l2')
plt.legend()
plt.title('Left Leg Joint Angles')

plt.subplot(3, 2, 4)
plt.plot(sol.t, sol.y[8, :], label='theta_r1')
plt.plot(sol.t, sol.y[9, :], label='theta_r2')
plt.legend()
plt.title('Right Leg Joint Angles')

plt.subplot(3, 2, 5)
plt.plot(sol.t, sol.y[10, :], label='phi_l')
plt.plot(sol.t, sol.y[11, :], label='phi_r')
plt.legend()
plt.title("Wheel Angles")

plt.tight_layout()
plt.show()

# Evaluate the constraints numerically to check
def eval_constraints(t, y):
    q_vals = y[:len(q)]
    q_dot_vals = y[len(q):]
    
    #Create input list for constraints
    args = [q_vals[i] for i in range(len(q))]
    args += [q_dot_vals[i] for i in range(len(q_dot))]
    
    #Evaluate constraints with subs
    cons_eval = [cons.subs(dict(zip(q+q_dot,args))) for cons in constraints]
    cons_eval_num = [float(cons_eq.subs(parameter_values)) for cons_eq in cons_eval]
    return cons_eval_num
    
# Eval at each time point
constraints_over_time = np.array([eval_constraints(t,y) for t, y in zip(sol.t, sol.y.T)])

# Plot the constraints
plt.figure()
for i in range(len(constraints)):
    plt.plot(sol.t, constraints_over_time[:, i], label=f'Constraint {i+1}')
plt.legend()
plt.title('Constraints over Time')
plt.xlabel('Time (s)')
plt.ylabel('Constraint Value')
plt.show()

### Linearization and Calculation of A and B Matrices

In [ ]:
def compute_eom_residuals(state_and_controls, L, q, q_dot, Q, constraints, lambdas, param_values):
    """
    Computes the residuals of the equations of motion (including constraints).
    This function is designed for use with numerical solvers.

    Args:
        state_and_controls: A NumPy array containing the state variables (q, q_dot),
                             control inputs (u), and Lagrange multipliers (lambdas).
        L: The Lagrangian.
        q: List of generalized coordinates.
        q_dot: List of generalized velocities.
        Q: List of generalized forces.
        constraints: List of constraint equations.
        lambdas: List of Lagrange multipliers.
        param_values: Dictionary mapping symbolic parameters to numerical values.

    Returns:
        A NumPy array of residuals (values of the left-hand side of the EOMs
        and constraint equations).
    """

    n = len(q)
    m = len(constraints)
    k = len(Q) - n   # Number of "control inputs"
    # Split state_and_controls
    current_q = state_and_controls[:n]
    current_q_dot = state_and_controls[n:2*n]
    current_controls =  state_and_controls[2*n : 2*n + k]
    current_lambdas = state_and_controls[2*n + k:]

    # Create substitution dictionaries
    q_subs = dict(zip(q, current_q))
    q_dot_subs = dict(zip(q_dot, current_q_dot))
    #Need to remake Q
    Q_temp = [0, 0, 0, 0, 0, 0, current_controls[0], current_controls[1], current_controls[2], current_controls[3], current_controls[4], current_controls[5]]

    Q_subs = dict(zip(Q, Q_temp))
    lambda_subs = dict(zip(lambdas, current_lambdas))

    subs_dict = {**q_subs, **q_dot_subs, **Q_subs, **lambda_subs, **param_values}

    # Evaluate equations of motion
    eom_values = [eom.subs(subs_dict) for eom in eoms]

    # Convert to NumPy array and flatten
    return np.array(eom_values, dtype=np.float64).flatten()

def compute_jacobian_numerically(func, x, *args):
    """
    Computes the Jacobian of a function numerically using central finite differences.

    Args:
        func: The function to compute the Jacobian of.
        x: The point at which to evaluate the Jacobian (NumPy array).
        *args: Additional arguments to pass to `func`.

    Returns:
        The numerically computed Jacobian matrix (NumPy array).
    """
    epsilon = 1e-8  # Step size for finite differences
    n = len(x)
    m = len(func(x, *args))  # Get number of outputs from func
    jacobian = np.zeros((m, n))

    for i in range(n):
        x_plus = x.copy()
        x_minus = x.copy()
        x_plus[i] += epsilon
        x_minus[i] -= epsilon
        f_plus = func(x_plus, *args)
        f_minus = func(x_minus, *args)
        jacobian[:, i] = (f_plus - f_minus) / (2 * epsilon)

    return jacobian
#Define function to combine states and controls for the residual
def combine_state_controls(q_vals, q_dot_vals, control_vals, lambda_vals):
    return np.concatenate([q_vals, q_dot_vals, control_vals, lambda_vals])

def split_state_controls(combined, n, k):
    q_vals = combined[:n]
    q_dot_vals = combined[n:2*n]
    control_vals = combined[2*n:2*n + k]
    lambda_vals = combined[2*n + k:]
    return q_vals, q_dot_vals, control_vals, lambda_vals
def compute_AB_matrices(L, q, q_dot, Q, constraints, lambdas, u, eq_state, param_values):
  """Computes the A and B matrices numerically."""
  n = len(q)       # number of generalized coordinates
  k = len(u)  # number of control inputs
  m = len(constraints) #number of constraints

  # Combine equilibrium state and zero controls
  eq_q = [eq_state[qi] for qi in q]   #numerical values
  eq_q_dot = [eq_state[qi] for qi in q_dot]
  eq_controls = [0] * k  # Zero controls at equilibrium
  # Initial guess for Lagrange multipliers 
  eq_lambdas = [0.0] * m

  #Initial state
  x0 = combine_state_controls(eq_q, eq_q_dot, eq_controls, eq_lambdas)

  # Find initial lambda values
  from scipy.optimize import fsolve
  # Define a function for fsolve
  def equations(initial_conditions):
      # Split initial conditions into q_ddot and lambda values
      initial_q_ddot, initial_lambdas = initial_conditions[:n], initial_conditions[n:]
      # Combine all values for subs: q, q_dot, q_ddot, lambdas, and control inputs
      combined_values = {**eq_state, **param_values}
      for i, q_dd in enumerate(q_ddot):
        combined_values[q_dd] = initial_q_ddot[i]
      for i, lm in enumerate(lambdas):
        combined_values[lm] = initial_lambdas[i]
      #also need to add zero controls
      for i, u_term in enumerate(u):
        combined_values[u_term] = 0

      # Evaluate eoms with substitutions
      evaluated_eoms = [eom.subs(combined_values) for eom in eoms]
    
      return evaluated_eoms

  # Call fsolve to find initial_q_ddot and initial_lambdas
  initial_conditions_guess = [0.0] * (n + m)  # Initial guess for q_ddot and lambdas
  solved_initial_conditions, info, ier, mesg = fsolve(equations, initial_conditions_guess, full_output=True)
  if ier != 1:
    print("Warning: fsolve did not converge:", mesg)
  eq_lambdas = solved_initial_conditions[n:] #update lambdas


  #Recalculate x0 with solved lambdas
  x0 = combine_state_controls(eq_q, eq_q_dot, eq_controls, n           )

  #Compute the A and B matrices
  A_numerical = compute_jacobian_numerically(compute_eom_residuals, x0, L, q, q_dot, Q, constraints, lambdas, param_values)
  B_numerical = A_numerical[:, 2*n:2*n + k] #B is part of the big jacobian
  A_numerical = A_numerical[:, :2*n]       #A is the rest of the big jacobian

  return A_numerical, B_numerical

# Example usage (assuming you have 'eoms', 'q', 'q_dot', 'Q', 'constraints', 'lambdas', 'u', 'eq_state', and 'param_values' defined)
# Create a dictionary for your parameter values
param_values = {
  m_c: 1.0, m_l1: 0.1, m_l2: 0.1, m_r1: 0.1, m_r2: 0.1, m_lw: 0.05, m_rw: 0.05,
  I_c: 0.1, I_l1: 0.01, I_l2: 0.01, I_r1: 0.01, I_r2: 0.01, I_lw: 0.0025, I_rw: 0.0025,
  L_1: 0.2, L_2: 0.2, r_w: 0.05, g: 9.81, lx: 0.1, ly: 0.1, lz: -0.1, rx: 0.1, ry: -0.1, rz: -0.1
}

A_num, B_num = compute_AB_matrices(L, q, q_dot, Q, constraints, lambdas, u, eq_state, param_values)
print("Numerical A Matrix:\n", A_num)
print("\nNumerical B Matrix:\n", B_num)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
def compute_A_matrix(q, q_dot, q_ddot, eoms, eq_state):
    n = len(q)
    A = sp.zeros(2*n, 2*n)
    
    # Solve for q̈ terms (rearrange eoms to get q̈ isolated)
    solved_q_ddot = sp.solve(eoms, q_ddot)
    
    # Upper-right block: derivative of q̇ wrt q̇ (identity matrix)
    for i in range(n):
        A[i, n+i] = 1
    
    # Lower blocks: derivatives of q̈ expressions
    for i in range(n):
        # Derivative with respect to q
        for j in range(n):
            A[n+i, j] = sp.diff(solved_q_ddot[q_ddot[i]], q[j]).subs(eq_state).subs(zero_control_state)
        
        # Derivative with respect to q̇
        for j in range(n):
            A[n+i, n+j] = sp.diff(solved_q_ddot[q_ddot[i]], q_dot[j]).subs(eq_state).subs(zero_control_state)
    
    return A

def compute_B_matrix(q, q_dot, q_ddot, u, eoms, eq_state):
    n = len(q)
    m = len(u)
    B = sp.zeros(2*n, m)
    
    # Solve for q̈ terms (rearrange eoms to get q̈ isolated)
    solved_q_ddot = sp.solve(eoms, q_ddot)
    
    # Lower blocks: derivatives of q̈ expressions
    for i in range(n):
        # Derivative with respect to u
        for j in range(m):
            B[n+i, j] = sp.diff(solved_q_ddot[q_ddot[i]], u[j]).subs(eq_state)
    
    return B


A_eq = compute_A_matrix(q, q_dot, q_ddot, eoms, eq_state)
B_eq = compute_B_matrix(q, q_dot, q_ddot, u, eoms, eq_state)

pprint(A_eq)
pprint(B_eq)